# Task description:

You need to predict new diseases for patients.

Train data contains medical information for 2018-2019 period and labels with new diseases discovered in 2019.

You are required to predict new diseases for 2020 using data for 2018-2019.

Background
Hierarchical condition category relies on ICD-10 coding to assign risk scores to patients. Each HCC is mapped to an ICD-10 code. Along with demographic factors (such as age and gender), insurance companies use HCC coding to assign patients a risk adjustment factor. It is very important to predict HCC coding to evaluate health risk of patients.

Every HCC can be mapped to several ICDs but not every ICD has a corresponding HCC. It is a one to many connection for HCC to ICDs.

We are providing ICD history of patients to predict their net new HCC coding for next year.

# Imports

In [1]:
import numpy as np 
import pandas as pd 
import pickle
import string
import re
import os

from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from scipy.spatial.distance import cosine
from datetime import datetime

import xgboost as xgb
from joblib import dump, load
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve, precision_score, recall_score

data_path = 'data/'

In [2]:
import warnings
# ignore mean warning for Nulls
warnings.filterwarnings(action='ignore', message='Mean of empty slice')

In [3]:
f = open(f"{data_path}icd_decode_di.pickle", "rb")
icd_decode_di = pickle.load(f)
print(len(icd_decode_di))
print(f"Get sample value: {icd_decode_di['K8020']}")

13506
Get sample value: Calculus of gallbladder without cholecystitis without obstruction


# Plan: 

1. Create statistical features using numerical data. 
2. Create embedding features using text data. 
3. Generate features for train (2018 year) and for test (2019 year).  
4. Split data on train and validation datasets.  
5. Train 30 xgboost models using training data for 2018 and labels for 2019.  
6. Choose threshold on validation dataset.  
7. Calculate AUC (mean) on validation dataset (65%).  
8. Train on full dataset using data for 2018 and labels for 2019.  
9. Make predictions for 2020 using selected thresholds and features generated for 2019.  

ToDo:
1. Create dynamic features - collect information how features have changed over time (for example, compare the first and last quarter).   
2. Mean targeting encoding: (1) - select the most common icd for each hcc and generate embeddings for these icds. (2) - Calculate distances between members' icd embeddings and (1).   
3. Tune xgboost params (especially, for the largest hcc classes). 1st step (use small number of n_estimators): max_depth, subsample, regularization;  2nd step (set larger number of n_estimators): add early stop; tune learning rate.  
4. Use the original xgboost instead of sklMultiOutputClassifierearn (the problem is that xgboost from sklearn cannot work with None values)

# Tools

Tools which used many times in notebook

### To work with DFs

In [4]:
def calc_date_diff(date_li):
    '''calculate difference between two string dates, return days'''
    
    regexp = '\d{4,4}\-\d{2,2}\-\d{2,2}'
    date_flag = all(
        [isinstance(c, str) and re.match(regexp,c) for c in date_li])
    if date_flag: 
        date_li = [datetime.strptime(c,'%Y-%m-%d') for c in date_li]
        date_diff = (date_li[1] - date_li[0]).days
        return date_diff if date_diff >= 0 else None
    else: 
        return None

def test_calc_date_diff(): 
    dates = ('2018-03-03', '2018-04-03')
    date_diff = calc_date_diff(dates)
    print(f'test_calc_date_diff:\ndates:{dates}\ndiff:{date_diff}')
    return True

print(test_calc_date_diff())

def calc_ndays_in_hosptl(df, clmn1:str, clmn2:str, new_col:str):
    '''calculate  difference between two string dates in df, 
    return df'''
    
    df = df.copy()
    df[new_col] = df[[clmn1, clmn2]].apply(
        lambda x: calc_date_diff(x), axis= 1).values
    return df

def get_year_slice_df(
    df,
    date_start:str,
    date_end:str,
    year:str): 
    '''get df slice in a specific year'''
    return df[df[[date_start, date_end]].apply(
        lambda x: all(c.split('-')[0] == year for c in x), axis= 1)].reset_index(drop= True)

test_calc_date_diff:
dates:('2018-03-03', '2018-04-03')
diff:31
True


### To work with text

In [5]:
pnct = string.punctuation
stop_words = set(stopwords.words('english'))

def remove_punctuation(word, punctuation= pnct):
    return re.sub(f'[{punctuation}]','', word) if isinstance(word, str) else None

def test_remove_punctuation(): 
    print('test remove punctuation')
    t = '!anemia,'
    print(t,'->', remove_punctuation(t))
    #return True

def prepare_text(text, stop_words= stop_words):
    if isinstance(text, str):
        # to lower register
        text = text.lower()
        # tokenize
        text = word_tokenize(text)
        # remove punctuation
        text = [remove_punctuation(w) for w in text]
        # filter out stop words
        text = [w for w in text if not w in stop_words and w != '']
        # stem (?) specific texts, can lose info..
        
        # Temp keep only words: 
        text = [w for w in text if w.isalpha()]
        
        return text
    else:
        return None

def test_prepare_text():
    print('test prepare text:')
    t = 'Displaced bicondylar fracture of left tibia, subsequent encounter for closed fracture with routine healing'
    print(t, '->\n', prepare_text(t))
    return True

print(test_prepare_text())

test prepare text:
Displaced bicondylar fracture of left tibia, subsequent encounter for closed fracture with routine healing ->
 ['displaced', 'bicondylar', 'fracture', 'left', 'tibia', 'subsequent', 'encounter', 'closed', 'fracture', 'routine', 'healing']
True


In [6]:
# can be one function 
def decode_and_collect_text_corpus(codes,
                                   decodes_di): 
    '''get codes, decode them to text, then tokenize and clean text.
       return list of lists
    '''
    num_missed = 0
    corpus = []
    for code in tqdm(codes): 
        decode = decodes_di.get(code)
        if pd.isnull(decode): 
            num_missed+=1
            continue
        if isinstance(decode, str): 
            corpus.append(prepare_text(decode))
    print(num_missed, len(corpus))
    return corpus

# not now:
def dis_decode_and_collect_text_corpus(
    code_df,
    decodes_di): 
    '''get codes and chronic flag,
       decode codes to text, then tokenize and clean text, add chronic flag.
       return list of lists
    '''
    num_missed= 0
    corpus = []
    for code, chronic_flag in tqdm(code_df.values): 
        decode = decodes_di.get(code)
        if pd.isnull(decode): 
            num_missed+=1
            continue
        if isinstance(decode, str): 
            decode = prepare_text(decode)
            if isinstance(chronic_flag, str) and chronic_flag != 'Unknown': 
                # add chronic flag to prepared text of icd 
                decode = decode + [chronic_flag.lower()]
            corpus.append(decode)
    print(num_missed, len(corpus))
    return corpus

In [7]:
def get_adm_icd_embedding(icd_code,
                          decodes_di,
                          w2v_model,
                          ): 
    '''get icd code and pre trained model.
    return embedding'''
    
    decode = decodes_di.get(icd_code)
    embngs_li = []
    missed_words = 0
    if isinstance(decode, str):
        clnd_tokens_li = prepare_text(decode)
        for w in clnd_tokens_li: 
            try: 
                embng = w2v_model.wv[w]
                embngs_li.append(embng)
            except Exception as e: 
                # print(e)
                missed_words+=1
        if len(embngs_li) > 0:
            return sum(embngs_li)/len(embngs_li)

# Feature generation

## Icd_code to embeddings
Get icd code from admissions and diseas data from 2018 to train word2vec model.  
Better embeddings achived using only on admissions data. 

In [8]:
admissions_df = pd.read_csv(
    os.path.join(data_path, 'admissions_data.csv'))
print(admissions_df.shape)
admissions_df.head(2)

(26479, 8)


,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission
0,AAoq6W9QR0G/kOfNZnoCPg==,2018-04-03,2018-04-03,OTH,NaN,No,N,NaN
1,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-04,2018-06-04,OTH,NaN,No,N,62.0


### Data from 2018

In [9]:
admissions_2018df = get_year_slice_df(
    df= admissions_df, 
    date_start= 'admission_date',
    date_end= 'discharge_date', 
    year= '2018'
  )

# save to encode feauters
admission_types_set = set(
    admissions_2018df.admission_type.unique()) 
print(f'Number admission types: {len(admission_types_set)}')

print(admissions_2018df.shape,
      admissions_2018df.member_id.nunique())
admissions_2018df.head()

Number admission types: 8
(7334, 8) 3552


,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission
0,AAoq6W9QR0G/kOfNZnoCPg==,2018-04-03,2018-04-03,OTH,NaN,No,N,NaN
1,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-04,2018-06-04,OTH,NaN,No,N,62.0
2,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-18,2018-06-18,OTH,NaN,No,N,14.0
3,AAoq6W9QR0G/kOfNZnoCPg==,2018-10-11,2018-10-11,OTH,NaN,No,N,115.0
4,AAoq6W9QR0G/kOfNZnoCPg==,2018-11-02,2018-11-02,OTH,NaN,No,N,22.0


### adm_icd_corpus

In [10]:
# collect unique icd_codes
adm_icd_code_2018 = admissions_2018df.icd_code.dropna().apply(
    lambda x: x.split(';')).explode('icd_code').drop_duplicates()
print(f'Unique icd_code in 2018: {len(adm_icd_code_2018)}')

Unique icd_code in 2018: 1175


In [11]:
# icd to tokens:
adm_icd_corpus_2018 = decode_and_collect_text_corpus(
    adm_icd_code_2018,
    icd_decode_di
    )
print(adm_icd_corpus_2018[0])

  0%|          | 0/1175 [00:00<?, ?it/s]

8 1167
['syncope', 'collapse']


In [12]:
# train model only on adm_icd_corpus_2018
adm_icd_wv_model = Word2Vec(
    adm_icd_corpus_2018,
    vector_size= 100,
    min_count=1,
    workers= 5,
    window = 2,
    seed = 1
)

In [13]:
# Fast check: distanse between v1 and v2 smaller then v1 and v3: 
v1 = get_adm_icd_embedding(
    'A048',
    icd_decode_di,
    adm_icd_wv_model)

v2 = get_adm_icd_embedding(
    'A049',
    icd_decode_di,
    adm_icd_wv_model)

v3 = get_adm_icd_embedding(
    'Z9989',
    icd_decode_di,
    adm_icd_wv_model)

print(
prepare_text(icd_decode_di['A048']),'\n',
prepare_text(icd_decode_di['A049']),'\n', 
prepare_text(icd_decode_di['Z9989'])
)

# unspecified - drop it
print(cosine(v1,v2), cosine(v1, v3))

['specified', 'bacterial', 'intestinal', 'infections'] 
 ['bacterial', 'intestinal', 'infection', 'unspecified'] 
 ['dependence', 'enabling', 'machines', 'devices']
0.3897094130516052 0.9311919212341309


## Features from admissions data

In [14]:
# funcs to get embdngs 
def combine_many_embdngs(
    x,
    decode_di:dict,
    vw_model,
    dim:int
): 
    '''get average vector from icd_code embedings. 
    return averaged vector'''
    
    embdngs_li = []
    for c in x: 
        if not pd.isnull(c):
            embdng = get_adm_icd_embedding(
                c,
                decode_di, 
                vw_model) 
            if isinstance(embdng, np.ndarray): 
                embdngs_li.append(embdng)
    l = len(embdngs_li)
    if l > 0:
        return sum(embdngs_li)/l
    else: 
        return [None] * dim

def generate_icd_embng_for_df(
    df, 
    id_column:str,
    emb_clmn:str,
    decode_di:dict, 
    wv_model, # trained model to get embdngs
    dim:int, # vector size
    emb_feature_name:str):
    '''group by id and generate avereged embedding. 
    return df with id and embeddings'''

    df = df[[id_column, emb_clmn]].dropna().copy()

    df[emb_clmn] = df[emb_clmn].str.split(';')
    df = df.explode(emb_clmn)

    df.drop_duplicates(
        inplace= True)
    df.reset_index(
        drop= True,
        inplace= True)
    
    # group by id
    df = df.groupby(id_column)[emb_clmn].apply(
        lambda x: combine_many_embdngs(
            x,
            decode_di,
            wv_model,
            dim
        )
    ).copy()
    
    df = df.reset_index()
    # adding embeddings columns
    new_cols = [f'{emb_feature_name}_{i}' for i in range(1,dim + 1)]
    df[new_cols] = pd.DataFrame(
        df[emb_clmn].to_list(), 
        index = df.index).values
    
    # some codes have no embeddings
    prev_num_rows = df.shape[0]
    df.dropna(inplace= True)
    print(f'Percent of Null embdngs: {(1 - prev_num_rows/df.shape[0]):.2f}')
    
    df.drop(emb_clmn, axis= 1, inplace= True)
    df.reset_index(drop= True, inplace= True)
    return df

In [15]:
adm_icd_embngs_2018df = generate_icd_embng_for_df(
    admissions_2018df,
    id_column = 'member_id',
    emb_clmn = 'icd_code', 
    decode_di = icd_decode_di, 
    wv_model = adm_icd_wv_model,
    dim = 100, 
    emb_feature_name = 'icd_embdng'
)

Percent of Null embdngs: -0.00


In [16]:
class Generate_Adm_Features: 
    '''Generate feares froom admissons data. return df'''
    
    def __init__(self,
                 df,
                 date_strat:str,
                 date_end:str,
                 year:str, 
                 adm_types_set:set, 
                 id_column: str, 
                 emb_clmn: str, 
                 decode_di: dict, 
                 wv_model, 
                 dim,
                 emb_feature_name
                ): 

        self.df = df
        self.date_strat = date_strat
        self.date_end = date_end
        self.year = year
        self.adm_types_set = adm_types_set 
        self.id_column = id_column
        self.emb_clmn = emb_clmn
        self.decode_di = decode_di
        self.wv_model = wv_model
        self.dim = dim
        self.emb_feature_name = emb_feature_name
        
    def adm_hosptl_day_agg_rules(self, x): 
        '''rules to aggregate days in hospital'''       

        d = {
            'num_apprnc_in_hosptl'   : x['admission_date'].count(),
            'sum_readmissions'       : x['enc_is_readmission'].sum(), 
            'sum_admission_transfer' : x['enc_admission_transfer'].sum(),
            
            'max_day_in_hosptl'    : x['day_in_hosptl'].max(),
            'mean_day_in_hosptl'   : x['day_in_hosptl'].mean(),
        }
        return pd.Series(d, index= d.keys())

    def adm_agg_hosptl_day_df(self, df):
        '''aggregate days in hospital'''
        
        return df.groupby('member_id').apply(
            self.adm_hosptl_day_agg_rules).reset_index()
    
    def generate_features(self): 
        
        # get data from specific date
        self.df = get_year_slice_df(
            self.df,
            self.date_strat,
            self.date_end, 
            self.year
          )
        
        if self.df.shape[0] == 0: 
            print(f'df shape equals 0')
            return self.df

        # calculate new feature - day in hospital
        self.df = calc_ndays_in_hosptl(
            df = self.df,
            clmn1 = self.date_strat,
            clmn2 = self.date_end,
            new_col = 'day_in_hosptl'
            )
        
        # encode words
        self.df['enc_is_readmission'] = self.df.is_readmission.replace(
            {'No':0, 'Yes':1})
        self.df['enc_admission_transfer'] = self.df.er_to_inp_admission_transfer.replace(
            {'N':0,'Y':1})
        
        # exclude unknown admission_type 
        self.df.admission_type = self.df.admission_type.apply(
            lambda x: x if x in self.adm_types_set else None)
        
        # get dummies from admission_type
        dummy_df = pd.get_dummies(
            self.df.set_index('member_id').admission_type, 
            prefix= 'adm_type').reset_index()
        
        # aggregate dummies
        dummy_df = dummy_df.groupby('member_id').sum()
    
        # generate and agg embeddings: 
        icd_embngs_df = generate_icd_embng_for_df(
            self.df,
            self.id_column,
            self.emb_clmn, 
            self.decode_di, 
            self.wv_model,
            self.dim, 
            self.emb_feature_name
        )
    
        # aggregate features 
        self.df = self.adm_agg_hosptl_day_df(self.df)
    
        # combine features 
        print(f'Dummies shape matched: {dummy_df.shape[0] == self.df.shape[0]}')
        
        self.df = self.df.merge(
            dummy_df,
            on = 'member_id', 
            how = 'inner'
        )
        
        self.df = self.df.merge(
            icd_embngs_df, 
            on = 'member_id', 
            how= 'left'
        )
        print(f'Unique id: {self.df.member_id.nunique()}, shape: {self.df.shape}')
        return self.df

In [17]:
adm_features_2018df = Generate_Adm_Features(
    df = admissions_df, 
    date_strat = 'admission_date', 
    date_end='discharge_date',
    year = '2018',
    adm_types_set = admission_types_set,
    id_column = 'member_id',
    emb_clmn = 'icd_code', 
    decode_di = icd_decode_di, 
    wv_model = adm_icd_wv_model,
    dim = 100, 
    emb_feature_name = 'icd_embdng').generate_features()

Percent of Null embdngs: -0.00
Dummies shape matched: True
Unique id: 3552, shape: (3552, 114)


In [18]:
adm_features_2019df = Generate_Adm_Features(
    df = admissions_df, 
    date_strat = 'admission_date', 
    date_end='discharge_date',
    year = '2019',
    adm_types_set = admission_types_set,
    id_column = 'member_id',
    emb_clmn = 'icd_code', 
    decode_di = icd_decode_di, 
    wv_model = adm_icd_wv_model,
    dim = 100, 
    emb_feature_name = 'icd_embdng').generate_features()

Percent of Null embdngs: -0.02
Dummies shape matched: True
Unique id: 7524, shape: (7524, 114)


In [19]:
print(set(adm_features_2018df.columns) - 
      set(adm_features_2019df.columns))

set()


In [20]:
adm_features_2018df.to_pickle(
    'kaggle/working/adm_features_2018df.pickle')

In [21]:
adm_features_2019df.to_pickle(
    'kaggle/working/adm_features_2019df.pickle')

# diseas_df

1. icd_chronic_or_acute != hcc_chronic_or_acute 3%

In [22]:
diseas_df = pd.read_csv(
    os.path.join(data_path, 'disease_data.csv'))
print(diseas_df.shape, diseas_df.member_id.nunique())
diseas_df.head(2)

(1022161, 6) 37071


,member_id,year_of_service,icd_code,icd_chronic_or_acute,hcc_code,hcc_chronic_or_acute
0,AAHLhKW7RYOEQIemWrh/1w==,2018,I739,Chronic,108.0,Chronic
1,AAHLhKW7RYOEQIemWrh/1w==,2018,M545,Unknown,NaN,NaN


In [23]:
diseas_df.hcc_chronic_or_acute.value_counts()

Chronic    317702
Acute        2584
Name: hcc_chronic_or_acute, dtype: int64

In [24]:
hcc_cat_unique = list(diseas_df[
    diseas_df.year_of_service == 2018].hcc_code.dropna().unique())
print(len(hcc_cat_unique))

64


In [25]:
class Generate_Hcc_Features():
    '''Generate hcc features for specific date.
    return df''' 
        
    def __init__(
        self, 
        df, 
        year:int, 
        clmn_id:str,
        hcc_catgs:str,
        embdng_clmn:str, 
        decode_di:dict,
        wv_model, 
        dim:int,
    ):
        
        self.df = df
        self.year = year
        self.clmn_id = clmn_id
        self.hcc_catgs = hcc_catgs
        self.embdng_clmn = embdng_clmn
        self.decode_di = decode_di
        self.wv_model = wv_model
        self.dim = dim        
        
    def left_merg(self, df1, df2):
        return df1.merge(df2, on = self.clmn_id, how= 'left')    
    
    def stats_icc_hcc(self, df): 
        d = {
            'dis_num_icd': df['icd_code'].dropna().count(),
            'dis_num_unique_icd' : df['icd_code'].nunique(),

            'dis_num_hcc': df['hcc_code'].dropna().count(),
            'dis_num_unique_hcc' : df['hcc_code'].nunique()
        }
        return pd.Series(d, index= d.keys())

    def agg_stats_icc_hcc(self, df):
            '''aggregate icc hcc info'''

            return df.groupby(self.clmn_id).apply(
                self.stats_icc_hcc).reset_index()
            
    def count_nflag(
        self, 
        df, 
        clmn_id:str,
        clmn_flag:str, 
        flag:str, 
        new_clmn_name:str):
        '''Count n flag for id. return df'''
        
        df = df[[clmn_id, clmn_flag]].copy()
        
        df = df.groupby(clmn_id)[clmn_flag].apply(
            lambda x: sum([True if isinstance(c,str) 
                                and (c == flag) else False for c in x]))
        df = df.reset_index()
        df.columns = [clmn_id, new_clmn_name]
        return df
    
    def generate_hcc_dummies(
        self,
        df):
        '''generate dummies variables'''
        
        df = df.copy()
        # get catgs only from train
        df['hcc_code'] = df['hcc_code'].apply(
            lambda x: x if x in self.hcc_catgs else None)

        df['hcc_code'] = df['hcc_code'].apply(
            lambda x: str(int(x)) if not pd.isnull(x) else x)

        hcc_dumm_df = pd.get_dummies(
                    df.set_index(self.clmn_id).hcc_code, 
                    prefix= 'dis_hcc_code').reset_index()
        
        hcc_dumm_df = hcc_dumm_df.groupby(
            self.clmn_id).sum().reset_index()
        
        return hcc_dumm_df
    
    def generate_features(self): 
        
        # get df on a specific date
        self.df = self.df[
            self.df.year_of_service == self.year].copy()
        
        self.df.reset_index(drop= True, inplace= True)
        
        # count statistics on icd, hcc codes
        stats_code_df = self.agg_stats_icc_hcc(self.df)
            
        # count flag
        n_hcc_chronic_df = self.count_nflag(
            df = self.df, 
            clmn_id = self.clmn_id,
            clmn_flag = 'hcc_chronic_or_acute',
            flag = 'Chronic',
            new_clmn_name = 'dis_hcc_nchronic'
        )     
        
        n_hcc_acute_df = self.count_nflag(
            df = self.df, 
            clmn_id = self.clmn_id,
            clmn_flag = 'hcc_chronic_or_acute',
            flag = 'Acute',
            new_clmn_name = 'dis_hcc_acute'
        )
        
        # hcc dummies
        hcc_dumm_df = self.generate_hcc_dummies(self.df)
        
        # icd embngs
        dis_icd_embngs_df = generate_icd_embng_for_df(
            df = self.df,
            id_column = self.clmn_id,
            emb_clmn = self.embdng_clmn, 
            decode_di = self.decode_di, 
            wv_model =  self.wv_model,
            dim = self.dim, 
            emb_feature_name = 'dis_icd_embdng'
        )             
        
        features_df = self.df[[self.clmn_id]].copy()
        features_df.drop_duplicates(inplace= True)
        features_df.reset_index(inplace= True, drop= True)
        
        # merge features
        features_df = self.left_merg(
            features_df,
            stats_code_df)
        
        features_df = self.left_merg(
            features_df,
            n_hcc_chronic_df)
        
        features_df = self.left_merg(
            features_df,
            n_hcc_acute_df)
        
        features_df = self.left_merg(
            features_df, 
            hcc_dumm_df)
        
        features_df = self.left_merg(
            features_df, 
            dis_icd_embngs_df)
        
        return features_df

In [26]:
dis_features_2018df = Generate_Hcc_Features(
    df = diseas_df, 
    year = 2018, 
    clmn_id = 'member_id', 
    hcc_catgs = hcc_cat_unique,
    embdng_clmn = 'icd_code',
    decode_di = icd_decode_di, 
    wv_model = adm_icd_wv_model, 
    dim = 100).generate_features()

Percent of Null embdngs: -0.00


In [27]:
print(dis_features_2018df.shape)
dis_features_2018df.head(3)

(27981, 171)


,member_id,dis_num_icd,dis_num_unique_icd,dis_num_hcc,dis_num_unique_hcc,dis_hcc_nchronic,dis_hcc_acute,dis_hcc_code_1,dis_hcc_code_10,dis_hcc_code_103,...,dis_icd_embdng_91,dis_icd_embdng_92,dis_icd_embdng_93,dis_icd_embdng_94,dis_icd_embdng_95,dis_icd_embdng_96,dis_icd_embdng_97,dis_icd_embdng_98,dis_icd_embdng_99,dis_icd_embdng_100
0,AAHLhKW7RYOEQIemWrh/1w==,2,2,1,1,1,0,0,0,0,...,0.003796,0.000803,0.004297,-0.000859,0.005743,0.000760,0.002740,0.000940,0.003500,0.000753
1,AAKW/PKhRva+ZS2B4s0AmA==,52,52,9,4,9,0,0,0,0,...,0.005001,0.002048,0.002092,0.000456,0.006698,0.003712,0.002229,-0.003114,0.002555,-0.001316
2,AAT/v3ExQAyfas8Ov5JevA==,2,2,2,2,2,0,0,0,0,...,0.005468,0.002126,0.001628,0.002052,0.005924,0.002430,0.003111,-0.003188,-0.000476,-0.000442


In [28]:
dis_features_2019df = Generate_Hcc_Features(
    df = diseas_df, 
    year = 2019, 
    clmn_id = 'member_id', 
    hcc_catgs = hcc_cat_unique,
    embdng_clmn = 'icd_code',
    decode_di = icd_decode_di, 
    wv_model = adm_icd_wv_model, 
    dim = 100).generate_features()

Percent of Null embdngs: -0.00


In [29]:
print(dis_features_2019df.shape)
dis_features_2019df.head()

(35998, 170)


,member_id,dis_num_icd,dis_num_unique_icd,dis_num_hcc,dis_num_unique_hcc,dis_hcc_nchronic,dis_hcc_acute,dis_hcc_code_1,dis_hcc_code_10,dis_hcc_code_103,...,dis_icd_embdng_91,dis_icd_embdng_92,dis_icd_embdng_93,dis_icd_embdng_94,dis_icd_embdng_95,dis_icd_embdng_96,dis_icd_embdng_97,dis_icd_embdng_98,dis_icd_embdng_99,dis_icd_embdng_100
0,AAHLhKW7RYOEQIemWrh/1w==,5,5,5,5,5,0,0,0,0,...,0.007962,0.001319,0.003897,-0.000160,0.010936,0.006829,0.000054,-0.005388,0.002292,0.001477
1,AAKW/PKhRva+ZS2B4s0AmA==,37,37,5,4,5,0,0,0,0,...,0.005513,0.001772,0.002458,-0.000057,0.007772,0.003809,0.003819,-0.003357,0.002762,-0.001229
2,AAT/v3ExQAyfas8Ov5JevA==,3,3,3,3,3,0,0,0,0,...,0.005503,0.001311,0.002141,0.001894,0.005488,0.001421,0.002808,-0.002017,0.000992,0.000300
3,AAlNgHwPRw2nNdCnsEKnyA==,10,10,0,0,0,0,0,0,0,...,0.001572,0.001982,0.001205,0.000619,0.005866,0.001888,0.000850,-0.002506,0.000680,-0.001813
4,AAoq6W9QR0G/kOfNZnoCPg==,20,20,3,3,3,0,0,0,0,...,0.003423,0.003001,0.003093,0.000863,0.005456,0.003895,0.001769,-0.003620,0.002052,-0.003192


In [30]:
print(
    set(dis_features_2018df.columns) - set(dis_features_2019df.columns))
print(
      set(dis_features_2019df.columns) - set(dis_features_2018df.columns))

{'dis_hcc_code_160'}
set()


In [31]:
dis_features_2019df['dis_hcc_code_160'] = 0
dis_features_2019df = dis_features_2019df[
    dis_features_2018df.columns].copy()

In [32]:
print(
    set(dis_features_2018df.columns) - set(dis_features_2019df.columns))
print(
      set(dis_features_2019df.columns) - set(dis_features_2018df.columns))

set()
set()


In [33]:
dis_features_2018df.to_pickle(
    'kaggle/working/dis_features_2018df.pickle')

In [34]:
dis_features_2019df.to_pickle(
    'kaggle/working/dis_features_2019df.pickle')

## labs_data

In [35]:
labs_df = pd.read_csv(os.path.join(data_path, 'labs_data.csv'))
print(labs_df.shape)
labs_df.head()

(5918612, 8)


,member_id,date_of_service,loinc_code,performed_test_name,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric
0,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,13457-7,LIPID PANEL WITH REFLEX TO DIRECT LDL,NaN,85000.0,0.0,99.0
1,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,17861-6,CALCIUM,NaN,9600.0,8.6,10.4
2,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,33914-3,EGFR NON-AFR. AMERICAN,NaN,91000.0,60.0,0.0
3,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,4544-3,HEMATOCRIT,NaN,42800.0,35.0,45.0
4,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,713-8,EOSINOPHILS,NaN,6800.0,0.0,0.0


In [36]:
class Generate_Labs_Features():
    '''Generate labs features.return df'''
    
    def __init__(
        self,
        df, 
        year: int
    ):
        self.df = df
        self.year = year
    
    def correct_test_value(self, df):
        '''Initial value seems to be from other scale. return df.'''
        df = df.copy()
        df['correct_res_val'] = df.result_value.apply(
            lambda x: x/1000 if not pd.isnull(x) else x).values
        return df

    def flag_res_val(self, x):
        '''Convert res test val to category'''
        
        res_val, lower_bound, upper_bound = x
        if res_val <= lower_bound:
            return 'lower'
        elif res_val >= upper_bound: 
            return 'higher'
        elif lower_bound < res_val < upper_bound:
            return 'normal'
        else:
            return None
    
    def add_test_result_clmn(self, df):
        '''Add new column with category for test value'''

        df = df.copy()
        df['test_result'] = df[[
            'correct_res_val', 
            'normal_low_value_numeric',
            'normal_high_value_numeric']].apply(
            lambda x: self.flag_res_val(x), axis= 1)
        return df

    def test_agg_rules(self, df): 
        d = {
            'n_test' : df['test_result'].count(),
            'n_normal_test': sum(df['test_result'] == 'normal'),
            'n_higher_test': sum(df['test_result'] == 'higher'),
            'n_lower_test': sum(df['test_result'] == 'lower'),
        }
        return pd.Series(d, index= d.keys())

    def agg_test_results(self, df):
            '''Aggregate test results. return df'''
            
            return df.groupby('member_id').apply(
                self.test_agg_rules).reset_index()    
        
    def generate_features(self): 
        
        # get specific date:
        self.df = self.df[self.df.date_of_service.apply(
            lambda x: True if isinstance(x,str) and 
                      x.split('-')[0] == str(self.year) else False)].copy()
        
        self.df.reset_index(
            drop= True,
            inplace= True
        )
        
        # add corrected test val
        self.df = self.correct_test_value(self.df)
        
        # add test result categories clmn:
        self.df = self.add_test_result_clmn(self.df)
        
        # aggregate 
        self.df = self.agg_test_results(self.df)
        print(f'Unique member_id: {self.df.member_id.nunique()}')
        return self.df

In [37]:
labs_feature_2018_df = Generate_Labs_Features(
    labs_df, 
    2018).generate_features()
print(labs_feature_2018_df.shape)
labs_feature_2018_df.head(3)

Unique member_id: 22468
(22468, 5)


,member_id,n_test,n_normal_test,n_higher_test,n_lower_test
0,++AdZ4gWTDSiSA9jUWHK2w==,68,46,15,7
1,++FdwsWnTo22gFqRFKYQKg==,145,113,28,4
2,++RzfxJ0SUiZqoeQTxNqwg==,161,119,40,2


In [38]:
labs_feature_2019_df = Generate_Labs_Features(
    labs_df, 
    2019).generate_features()
print(labs_feature_2019_df.shape)
labs_feature_2019_df.head(3)

Unique member_id: 23668
(23668, 5)


,member_id,n_test,n_normal_test,n_higher_test,n_lower_test
0,++AdZ4gWTDSiSA9jUWHK2w==,113,70,28,15
1,++FdwsWnTo22gFqRFKYQKg==,96,75,19,2
2,++RzfxJ0SUiZqoeQTxNqwg==,97,72,23,2


In [39]:
labs_feature_2018_df.to_pickle(
    'kaggle/working/labs_feature_2018_df.pickle')
labs_feature_2019_df.to_pickle(
    'kaggle/working/labs_feature_2019_df.pickle')

## patients_data

In [40]:
patients_df =  pd.read_csv(os.path.join(data_path, 'patients_data.csv'))
print(patients_df.shape)
patients_df.head()

(26559, 9)


,member_id,patient_age,patient_gender,dual_status,insurance_company,insurance_type,pbp_type,county,city
0,AAHLhKW7RYOEQIemWrh/1w==,72,M,DYoOR0bMMk3HRSo8sSPjUw==,GUD2qR6A4Wd2L73mE5cXSw==,0EM/AFr1Il2Z8J30Yadekw==,0EM/AFr1Il2Z8J30Yadekw==,UejZIiyN0zMFVlUbJl9WMw==,JcyeVCRLiLD21PlyRx0v3w==
1,AAKW/PKhRva+ZS2B4s0AmA==,72,M,tMEZlLbu7jT3s3qXZC+gWQ==,rOCadimmYUC//TJHZzkvRg==,0EM/AFr1Il2Z8J30Yadekw==,a85jcyRlCtomarSMXXjFbg==,1S4k4hzgPOiWaL9v6ChejA==,btsrhf68Jq+yI/2V8QTMMw==
2,AAT/v3ExQAyfas8Ov5JevA==,75,F,tMEZlLbu7jT3s3qXZC+gWQ==,rOCadimmYUC//TJHZzkvRg==,0EM/AFr1Il2Z8J30Yadekw==,9Ro3ya9jPvS3mg+7vLpsgg==,1S4k4hzgPOiWaL9v6ChejA==,btsrhf68Jq+yI/2V8QTMMw==
3,AAoq6W9QR0G/kOfNZnoCPg==,62,M,cxTcUuoRUVTg+SSt7OlnXg==,hdbB94EoBNp2gZ5k2t6eLw==,0EM/AFr1Il2Z8J30Yadekw==,9Ro3ya9jPvS3mg+7vLpsgg==,EvFxDbLnPKOa13Yw9vDU4A==,xg5tPYYDxhRt10rlHSQG3A==
4,AAu2+UMHSpK6gVYffGi4pw==,54,F,tMEZlLbu7jT3s3qXZC+gWQ==,rOCadimmYUC//TJHZzkvRg==,0EM/AFr1Il2Z8J30Yadekw==,0EM/AFr1Il2Z8J30Yadekw==,EvFxDbLnPKOa13Yw9vDU4A==,yjYhcMttIqpWhMY2PuP8Sg==


In [41]:
patients_df.insurance_type.value_counts()

0EM/AFr1Il2Z8J30Yadekw==    26324
BMa1lHUXGi0OtaI8j5f5Xg==      235
Name: insurance_type, dtype: int64

In [42]:
# dictionaries to decode:
unique_ins_comp = patients_df.insurance_company.unique()
l = len(unique_ins_comp)
ins_di = {key:val for key, val in zip(
    unique_ins_comp, [f'ins_cmp_{i}' for i in range(l)])}
print(ins_di,'\n')

unique_ins_type = patients_df.insurance_type.unique()
l_type = len(unique_ins_type)
ins_type_di = {key:val for key, val in zip(
    unique_ins_type, [i for i in range(l_type)])}
print(ins_type_di,'\n')

unique_ins_pbp_type = patients_df.pbp_type.dropna().unique()
l_pbp = len(unique_ins_pbp_type)
print(l_pbp)
pbp_type_di = {key:val for key, val in zip(
    unique_ins_pbp_type, [f'pbp_type_{i}' for i in range(l_pbp)])}
print(pbp_type_di)

{'GUD2qR6A4Wd2L73mE5cXSw==': 'ins_cmp_0', 'rOCadimmYUC//TJHZzkvRg==': 'ins_cmp_1', 'hdbB94EoBNp2gZ5k2t6eLw==': 'ins_cmp_2', 'mbxR1g+DHtUymXSCxRReng==': 'ins_cmp_3', 'FAaLRCWN+MuKARhT45hR7Q==': 'ins_cmp_4', 'ICZEkLxJSG8+Qs1o0U8shg==': 'ins_cmp_5', 'GKh4tI8VqJiQpTt6Gs0mwQ==': 'ins_cmp_6', 'Acw2uqWLh3hTK4o304WLNA==': 'ins_cmp_7', '73vVoLvI35jb+tqju1bH9Q==': 'ins_cmp_8', '4tjrOgFYVbQKQ3QVrdn1mQ==': 'ins_cmp_9', 'qw1xA4ALdG31N8SGQPrKLQ==': 'ins_cmp_10', 'vkyNNake+rB0q98wMTqVNg==': 'ins_cmp_11'} 

{'0EM/AFr1Il2Z8J30Yadekw==': 0, 'BMa1lHUXGi0OtaI8j5f5Xg==': 1} 

10
{'0EM/AFr1Il2Z8J30Yadekw==': 'pbp_type_0', 'a85jcyRlCtomarSMXXjFbg==': 'pbp_type_1', '9Ro3ya9jPvS3mg+7vLpsgg==': 'pbp_type_2', 'uDV6R83FBSQjvKOyPHzpAA==': 'pbp_type_3', 'Zoxw6j8UzcAa3tUJL9KcoA==': 'pbp_type_4', 'dB1YVE0we/hjr8WseWV0JA==': 'pbp_type_5', 'BMa1lHUXGi0OtaI8j5f5Xg==': 'pbp_type_6', 'DvsIkFzyFga4wjMyFU/zAw==': 'pbp_type_7', 'cIzGIavYZ2EHbBRNYdOzFQ==': 'pbp_type_8', 'BqjJ3k6sAFFJIUcVOtfzMw==': 'pbp_type_9'}


In [43]:
for clmn in patients_df.columns[1:]:
    print(f'{clmn} nunique vals: {patients_df[clmn].nunique()}')

patient_age nunique vals: 83
patient_gender nunique vals: 2
dual_status nunique vals: 9
insurance_company nunique vals: 12
insurance_type nunique vals: 2
pbp_type nunique vals: 10
county nunique vals: 48
city nunique vals: 499


In [44]:
class Generate_Patients_features():
    '''Generate patients features. return df'''
    
    def __init__(
        self, 
        df,):
        self.df = df
        
    def generate(self):
        
        self.df['encode_patient_gender'] = self.df.patient_gender.apply(
            lambda x: 1 if x == 'M' else 0)
        
        self.df['enc_ins_comp'] = self.df.insurance_company.apply(
            lambda x: ins_di.get(x))
        
        self.df['enc_ins_type'] = self.df.insurance_type.apply(
            lambda x: ins_type_di.get(x))
        
        self.df['enc_ins_pbp'] = self.df.pbp_type.apply(
            lambda x: pbp_type_di.get(x))
        
        ins_comp_dummy = pd.get_dummies(
            self.df.set_index('member_id').enc_ins_comp).reset_index()
        
        pbp_dummy = pd.get_dummies(
            self.df.set_index('member_id').enc_ins_pbp).reset_index()        
                
        feature_clmns = ['member_id',
                         'encode_patient_gender',
                         'enc_ins_type'
                        ]
        # merge dummies
        feature_df = self.df[feature_clmns].merge(
            ins_comp_dummy, 
            on = 'member_id'
        )
        
        feature_df = feature_df.merge(
            pbp_dummy, 
            on = 'member_id'
        )
            
        return feature_df

In [45]:
patients_feature_df = Generate_Patients_features(
    patients_df).generate()
print(patients_feature_df.shape)
patients_feature_df.head(3)

(26559, 25)


,member_id,encode_patient_gender,enc_ins_type,ins_cmp_0,ins_cmp_1,ins_cmp_10,ins_cmp_11,ins_cmp_2,ins_cmp_3,ins_cmp_4,...,pbp_type_0,pbp_type_1,pbp_type_2,pbp_type_3,pbp_type_4,pbp_type_5,pbp_type_6,pbp_type_7,pbp_type_8,pbp_type_9
0,AAHLhKW7RYOEQIemWrh/1w==,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,AAKW/PKhRva+ZS2B4s0AmA==,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,AAT/v3ExQAyfas8Ov5JevA==,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [46]:
patients_feature_df.to_pickle(
    'kaggle/working/patients_feature_df.pickle')

## prescription_data

In [47]:
prescriptions_df = pd.read_csv(os.path.join(
    data_path, 'prescriptions_data.csv'))
print(prescriptions_df.shape)

prescriptions_df.days_supply= prescriptions_df.days_supply.apply(
    lambda x: int(x) if not pd.isnull(x) else x)

# execute drug name
prescriptions_df['short_drug_name'] = prescriptions_df.drug_name.apply(
    lambda x: x.split()[0] if isinstance(x,str) else x)

# clean days_supply; metric_quantity
prescriptions_df['clnd_days_supply'] = prescriptions_df.days_supply.apply(
    lambda x: None if pd.isnull(x) or (x < 0) or (x > 364) else x)
prescriptions_df['clnd_metric_quantity'] = prescriptions_df.metric_quantity.apply(
    lambda x: None if pd.isnull(x) or (x < 0) or (x > 364) else x)

# check percent of Null
ds_null1 = prescriptions_df.days_supply.isnull().mean()
ds_null2 = prescriptions_df.clnd_days_supply.isnull().mean()

mq_null1 = prescriptions_df.metric_quantity.isnull().mean()
mq_null2 = prescriptions_df.clnd_metric_quantity.isnull().mean()

print('Columns changes after cleaning:')
print(f'days_supply percent of null:{ds_null1:.2} -> {ds_null2:.2}')
print(f'days_metric_quantity of null:{mq_null1:.2} -> {mq_null2:.2}')

prescriptions_df.head()

(1740855, 7)
Columns changes after cleaning:
days_supply percent of null:0.012 -> 0.04
days_metric_quantity of null:0.0 -> 0.016


,member_id,date_filled,ndc_number,days_supply,drug_name,metric_quantity,new_or_refill,short_drug_name,clnd_days_supply,clnd_metric_quantity
0,AAHLhKW7RYOEQIemWrh/1w==,2018-01-13,60505013400,60.0,CYCLOSPORINE 100 MG CAPS@100 M,60,N,CYCLOSPORINE,60.0,60.0
1,AAHLhKW7RYOEQIemWrh/1w==,2018-01-16,62175013643,90.0,OMEPRAZOLE DR 40 MG CAPS@40 MG,90,N,OMEPRAZOLE,90.0,90.0
2,AAHLhKW7RYOEQIemWrh/1w==,2018-02-06,50742013510,90.0,ALLOPURINOL 100 MG TABLE@100 M,90,N,ALLOPURINOL,90.0,90.0
3,AAHLhKW7RYOEQIemWrh/1w==,2018-02-14,60505267108,90.0,ATORVASTATIN 80 MG TABLE@80 MG,90,N,ATORVASTATIN,90.0,90.0
4,AAHLhKW7RYOEQIemWrh/1w==,2018-02-20,43547039910,14.0,CYCLOBENZAPRINE 5 MG TAB@5 MG,14,N,CYCLOBENZAPRINE,14.0,14.0


### Drug names embeddings

In [48]:
# get data from 2018 
prescriptions_2018df = prescriptions_df[prescriptions_df.date_filled.apply(
    lambda x: x.split('-')[0] == '2018')].reset_index(drop= True).copy()

# train model on 2018 drugs name corpus
short_drug_names = prescriptions_2018df.short_drug_name.drop_duplicates().dropna()
short_drug_names = short_drug_names.values
drug_corpus = [row.split(',') for row in short_drug_names]

# drugs name to vectors: 
drugs_model = Word2Vec(
    drug_corpus,vector_size= 50,
    min_count=1,
    workers=3,
    window = 1)

# embeddings dict
drg_emdngs = {}
missed_drugs= []
for short_name in short_drug_names:
    try:
        drg_emdngs[short_name] = drugs_model.wv[short_name]
    except: 
        missed_drugs.append(short_name)
        continue
print(f'Missed drugs names {len(missed_drugs)}')

Missed drugs names 3


In [49]:
# Example
v1 = drugs_model.wv['CYCLOSPORINE']
v2 = drugs_model.wv['OMEPRAZOLE']
v3 = drugs_model.wv['METOPROLOL']

w1 = drugs_model.wv['OMEPRAZOLE']
w2 = drugs_model.wv['ALLOPURINOL']
w3 = drugs_model.wv['METOPROLOL']

k1 = drugs_model.wv['ATORVASTATIN']
k2 = drugs_model.wv['CYCLOBENZAPRINE']

v = (v1+v2+v3)/3
w = (w1+w2+w3)/3
k = (k1+k2)/2

# the distance is greater if the drug vectors do not intersect
print(cosine(v,w))
print(cosine(k,v))
print(cosine(k,w))

0.3360474109649658
0.9170492589473724
1.0300169941037893


In [50]:
def get_embdngs(word_list, embdngs_di:dict): 
    all_embdngs = []
    for w in word_list: 
        embdng = embdngs_di.get(w, None)
        if isinstance(embdng, np.ndarray): 
            all_embdngs.append(embdng)
    l = len(all_embdngs) 
    if l == 0: 
        return None
    else: 
        return sum(all_embdngs)/l
    
def make_drug_embdngs_df(df,
                         id_clmn:str,
                         short_name_clmn:str,
                         embdngs_di:dict):
    
    # make embeddings for all clients drugs:
    df= df.groupby(id_clmn)[short_name_clmn].apply(
        lambda x: get_embdngs(x, embdngs_di))
    
    # serie to df:
    df = df.reset_index().dropna()
    df.reset_index(drop= True, inplace= True)
    
    # add new clmns:
    new_cols = [f'drag_name_{i}' for i in range(1,51)]
    df[new_cols] = pd.DataFrame(
        df.short_drug_name.to_list(),  
        index = df.index)
    
    # drop clmn with lists
    df.drop(
        'short_drug_name',
        axis=1,
        inplace= True
    )
    return df

In [51]:
drug_emdb_2018df = make_drug_embdngs_df(
    prescriptions_2018df,
    'member_id',
    'short_drug_name',
    drg_emdngs)

print(drug_emdb_2018df.shape)
drug_emdb_2018df.head(3)

(24919, 51)


,member_id,drag_name_1,drag_name_2,drag_name_3,drag_name_4,drag_name_5,drag_name_6,drag_name_7,drag_name_8,drag_name_9,...,drag_name_41,drag_name_42,drag_name_43,drag_name_44,drag_name_45,drag_name_46,drag_name_47,drag_name_48,drag_name_49,drag_name_50
0,++AdZ4gWTDSiSA9jUWHK2w==,-0.000655,-0.005604,0.008281,-0.000421,0.005794,-0.002235,-0.009141,-0.003223,-0.011817,...,-0.001491,0.000305,-0.003291,0.000837,-0.004193,-0.002608,-0.003282,0.003815,0.001568,-0.009516
1,++FdwsWnTo22gFqRFKYQKg==,-0.002602,-0.007564,0.007960,-0.002434,0.001990,-0.002928,-0.004668,0.003876,0.000125,...,-0.001616,0.001619,-0.002885,-0.003517,-0.005564,-0.002619,-0.006924,0.001489,0.001130,0.005472
2,++WtCm5CR+Gp9NGIYwJPhg==,0.007999,-0.002156,-0.004647,0.006226,0.002697,0.005112,-0.009143,0.003842,-0.007334,...,-0.000190,0.001535,0.000725,0.006978,-0.000865,0.004017,0.001586,-0.004177,0.000272,0.011009


In [52]:
# get data from 2019 
prescriptions_2019df = prescriptions_df[prescriptions_df.date_filled.apply(
    lambda x: x.split('-')[0] == '2019')].reset_index(drop= True).copy()

drug_emdb_2019df = make_drug_embdngs_df(
    prescriptions_2019df,
    'member_id',
    'short_drug_name',
    drg_emdngs)

print(drug_emdb_2019df.shape)
drug_emdb_2019df.head(3)

(25877, 51)


,member_id,drag_name_1,drag_name_2,drag_name_3,drag_name_4,drag_name_5,drag_name_6,drag_name_7,drag_name_8,drag_name_9,...,drag_name_41,drag_name_42,drag_name_43,drag_name_44,drag_name_45,drag_name_46,drag_name_47,drag_name_48,drag_name_49,drag_name_50
0,++AdZ4gWTDSiSA9jUWHK2w==,0.000378,-0.005247,0.007958,0.000251,0.006193,-0.000645,-0.008651,-0.002998,-0.012503,...,-0.002761,0.001480,-0.002162,0.003334,-0.006418,-0.003533,-0.003091,0.002827,0.002612,-0.011018
1,++FdwsWnTo22gFqRFKYQKg==,0.001110,-0.007972,0.009315,-0.006309,0.003716,-0.004656,-0.004758,0.002460,-0.001021,...,-0.004294,0.002544,-0.002692,-0.005732,-0.008873,-0.002794,-0.003716,-0.000338,0.004517,0.005463
2,++RzfxJ0SUiZqoeQTxNqwg==,0.011226,-0.003183,0.002463,-0.004221,0.015575,0.004628,0.000420,-0.003185,-0.014119,...,-0.002879,-0.017200,0.003146,-0.012482,-0.009709,-0.019637,-0.003806,0.005690,-0.017957,0.017797


### Calculate statistics on other features

ToDo: 
* dynamic features: how changed drugs and amount

In [53]:
class make_drugs_statistic_features_df:
    ''''''
    def __init__(self, df): 
        self.df = df
        
        # total statistics
        
        # days_supply
        self.ttl_ds_mean = self.df.groupby(
            'member_id').clnd_days_supply.mean().mean()
        
        self.ttl_ds_median = self.df.clnd_days_supply.median()
        
        self.ttl_ds_max = self.df.clnd_days_supply.max()
        
        # metric_quantity
        self.ttl_mq_mean = self.df.groupby(
            'member_id').clnd_metric_quantity.mean().mean()
        
        self.ttl_mq_median = self.df.clnd_metric_quantity.median()
        
        self.ttl_mq_max = self.df.clnd_metric_quantity.max()
        
        # ndc_number
        self.ttl_ndc_number_mean = self.df.groupby(
            'member_id').ndc_number.count().mean()
        
        self.ttl_ndc_number_unique_mean = self.df.groupby(
            'member_id').ndc_number.nunique().mean()
        
        
    def new_to_relif_ratio(self, clmn):
        vc = clmn.dropna().value_counts()
        if set(['N','R']) == set(vc.index):
            return vc['N']/vc['R'] if vc['R'] > 0 else None
        else: 
            return None

    def agg_rules(self, x): 
        d= {
            # ndc_number
            'count_ndc_number'       : x['ndc_number'].count(),
            'nunique_ndc_number'     : x['ndc_number'].nunique(),
            'rel_count_ndc_number'   : x['ndc_number'].count()/self.ttl_ndc_number_mean,
            'rel_nunique_ndc_number' : x['ndc_number'].nunique()/self.ttl_ndc_number_unique_mean,

            # days_supply 
            'mean_days_supply'  : x['clnd_days_supply'].mean(), # can be variable
            'std_days_supply'   : x['clnd_days_supply'].std(),
            'median_days_supply': x['clnd_days_supply'].median(),
            'max_days_supply'   : x['clnd_days_supply'].max(),
            'min_days_supply'   : x['clnd_days_supply'].min(),

            # relative days_supply to total 
            'rel_mean_days_supply'  : x['clnd_days_supply'].mean()/self.ttl_ds_mean,
            'rel_median_days_supply': x['clnd_days_supply'].median()/self.ttl_ds_median,
            'rel_max_days_supply'   : x['clnd_days_supply'].max()/self.ttl_ds_max,

            # metric_quantity
            'mean_metric_quantity'  : x['clnd_metric_quantity'].mean(),
            'std_metric_quantity'   : x['clnd_metric_quantity'].std(),
            'median_metric_quantity': x['clnd_metric_quantity'].median(),
            'max_metric_quantity'   : x['clnd_metric_quantity'].max(),
            'min_metric_quantity'   : x['clnd_metric_quantity'].min(),
            
            # relative metric_quantity to total 
            'rel_mean_metric_quantity'  : x['clnd_metric_quantity'].mean()/self.ttl_mq_mean,
            'rel_median_metric_quantity': x['clnd_metric_quantity'].median()/self.ttl_mq_median,
            'rel_max_metric_quantity'   : x['clnd_metric_quantity'].max()/self.ttl_mq_max,
            
            # new_or_refill
            'ratio_new_to_refill'       : self.new_to_relif_ratio(x['new_or_refill'])
        }

        return pd.Series(d, index= d.keys())
    
    def group_and_agg_df(self):
        return self.df.groupby('member_id').apply(
            self.agg_rules).reset_index()

In [54]:
stats_presrp_2018df = make_drugs_statistic_features_df(
    prescriptions_2018df).group_and_agg_df()
print(stats_presrp_2018df.shape)
stats_presrp_2018df.head(3)

(24921, 22)


,member_id,count_ndc_number,nunique_ndc_number,rel_count_ndc_number,rel_nunique_ndc_number,mean_days_supply,std_days_supply,median_days_supply,max_days_supply,min_days_supply,...,rel_max_days_supply,mean_metric_quantity,std_metric_quantity,median_metric_quantity,max_metric_quantity,min_metric_quantity,rel_mean_metric_quantity,rel_median_metric_quantity,rel_max_metric_quantity,ratio_new_to_refill
0,++AdZ4gWTDSiSA9jUWHK2w==,18.0,7.0,0.561346,0.523014,81.944444,23.833916,90.0,90.0,5.0,...,0.250000,101.222222,50.097030,90.0,180.0,6.0,1.379081,1.0,0.494505,0.500000
1,++FdwsWnTo22gFqRFKYQKg==,64.0,16.0,1.995895,1.195460,40.625000,36.570654,30.0,260.0,6.0,...,0.722222,57.031250,46.369109,45.0,180.0,7.0,0.777010,0.5,0.494505,0.422222
2,++WtCm5CR+Gp9NGIYwJPhg==,8.0,8.0,0.249487,0.597730,78.875000,31.466252,90.0,90.0,1.0,...,0.250000,67.625000,33.733991,90.0,90.0,1.0,0.921343,1.0,0.247253,7.000000


In [55]:
prescr_features_2018df = prescriptions_2018df[
    ['member_id']].drop_duplicates().reset_index(drop= True).copy()
print(prescr_features_2018df.shape)

prescr_features_2018df = prescr_features_2018df.merge(
    drug_emdb_2018df, 
    on = 'member_id', 
    how= 'left'
)
print(prescr_features_2018df.shape)

prescr_features_2018df = prescr_features_2018df.merge(
    stats_presrp_2018df, 
    on = 'member_id', 
    how= 'left'
)
print(prescr_features_2018df.shape)

(24921, 1)
(24921, 51)
(24921, 72)


In [56]:
stats_presrp_2019df = make_drugs_statistic_features_df(
    prescriptions_2019df).group_and_agg_df()
print(stats_presrp_2019df.shape)
stats_presrp_2019df.head(3)

(25877, 22)


,member_id,count_ndc_number,nunique_ndc_number,rel_count_ndc_number,rel_nunique_ndc_number,mean_days_supply,std_days_supply,median_days_supply,max_days_supply,min_days_supply,...,rel_max_days_supply,mean_metric_quantity,std_metric_quantity,median_metric_quantity,max_metric_quantity,min_metric_quantity,rel_mean_metric_quantity,rel_median_metric_quantity,rel_max_metric_quantity,ratio_new_to_refill
0,++AdZ4gWTDSiSA9jUWHK2w==,17.0,7.0,0.471269,0.466467,84.764706,21.585671,90.0,90.0,1.0,...,0.25,105.941176,47.434785,90.0,180.0,1.0,1.421195,1.000000,0.497238,1.12500
1,++FdwsWnTo22gFqRFKYQKg==,60.0,12.0,1.663303,0.799658,44.833333,27.539904,30.0,90.0,25.0,...,0.25,45.900000,32.016786,30.0,90.0,11.0,0.615746,0.333333,0.248619,0.22449
2,++RzfxJ0SUiZqoeQTxNqwg==,2.0,2.0,0.055443,0.133276,90.000000,0.000000,90.0,90.0,90.0,...,0.25,90.000000,0.000000,90.0,90.0,90.0,1.207345,1.000000,0.248619,NaN


In [57]:
prescr_features_2019df = prescriptions_2019df[
    ['member_id']].drop_duplicates().reset_index(drop= True).copy()
print(prescr_features_2019df.shape)

prescr_features_2019df = prescr_features_2019df.merge(
    drug_emdb_2019df, 
    on = 'member_id', 
    how= 'left'
)
print(prescr_features_2019df.shape)

prescr_features_2019df = prescr_features_2019df.merge(
    stats_presrp_2019df, 
    on = 'member_id', 
    how= 'left'
)
print(prescr_features_2019df.shape)

(25877, 1)
(25877, 51)
(25877, 72)


In [58]:
prescr_features_2018df.to_pickle(
    'kaggle/working/prescr_features_2018df.pickle')

In [59]:
prescr_features_2019df.to_pickle(
    'kaggle/working/prescr_features_2019df.pickle')

## sample_submission

In [60]:
sample_submission_df = pd.read_csv(
    os.path.join(data_path, 'sample_submission.csv'))
print(sample_submission_df.shape)
sample_submission_df.head()

(26559, 31)


,member_id,hcc_9,hcc_10,hcc_11,hcc_12,hcc_18,hcc_19,hcc_21,hcc_22,hcc_23,...,hcc_88,hcc_96,hcc_103,hcc_108,hcc_111,hcc_112,hcc_124,hcc_137,hcc_138,hcc_161
0,AAHLhKW7RYOEQIemWrh/1w==,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3
1,AAKW/PKhRva+ZS2B4s0AmA==,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3
2,AAT/v3ExQAyfas8Ov5JevA==,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3
3,AAoq6W9QR0G/kOfNZnoCPg==,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3
4,AAu2+UMHSpK6gVYffGi4pw==,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3


In [61]:
# Check ouput format:
req_columns = sample_submission_df.columns
print(f'Required columns: {list(req_columns)},\nclmns number: {len(req_columns)}')
print(sample_submission_df.member_id.nunique())

Required columns: ['member_id', 'hcc_9', 'hcc_10', 'hcc_11', 'hcc_12', 'hcc_18', 'hcc_19', 'hcc_21', 'hcc_22', 'hcc_23', 'hcc_40', 'hcc_47', 'hcc_48', 'hcc_51', 'hcc_52', 'hcc_55', 'hcc_59', 'hcc_75', 'hcc_79', 'hcc_84', 'hcc_85', 'hcc_88', 'hcc_96', 'hcc_103', 'hcc_108', 'hcc_111', 'hcc_112', 'hcc_124', 'hcc_137', 'hcc_138', 'hcc_161'],
clmns number: 31
26559


## train_labels

In [62]:
train_labels_df = pd.read_csv(
    os.path.join(data_path, 'train_labels.csv'))
print(train_labels_df.shape)
train_labels_df.head()

(26559, 31)


,member_id,hcc_9,hcc_10,hcc_11,hcc_12,hcc_18,hcc_19,hcc_21,hcc_22,hcc_23,...,hcc_88,hcc_96,hcc_103,hcc_108,hcc_111,hcc_112,hcc_124,hcc_137,hcc_138,hcc_161
0,c5xNoBb+QoCII8LRiNnScg==,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,0,0,1,0
1,rofIXF8fT9CAAtouT8yPSQ==,0,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,1,1,0
2,7Rs4fvk6TJG2rYLiXGLFnQ==,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3Obni1mlS1appaOGlL5DJg==,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,gZTlsjUkTROZtdaTeiPlhQ==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
print(f'member_id nunique: {train_labels_df.member_id.nunique()}')

member_id nunique: 26559


# Collect train data set

In [64]:
patients_df = pd.read_pickle(
    'kaggle/working/patients_feature_df.pickle')

adm_features_2018df = pd.read_pickle(
    'kaggle/working/adm_features_2018df.pickle')

dis_features_2018df = pd.read_pickle(
    'kaggle/working/dis_features_2018df.pickle')

labs_feature_2018df = pd.read_pickle(
    'kaggle/working/labs_feature_2018_df.pickle')

prescr_features_2018df = pd.read_pickle(
    'kaggle/working/prescr_features_2018df.pickle')

In [65]:
train_df = train_labels_df[['member_id']].copy()

for df in [adm_features_2018df, dis_features_2018df, 
           labs_feature_2018df, prescr_features_2018df]:
    
    train_df = train_df.merge(
        df, 
        on = 'member_id', 
        how= 'left'
    )
    print(train_df.shape)
    
train_df.to_pickle('kaggle/working/train_df.pickle')

(26559, 114)
(26559, 284)
(26559, 288)
(26559, 359)


In [66]:
adm_features_2019df = pd.read_pickle(
    'kaggle/working/adm_features_2019df.pickle')

dis_features_2019df = pd.read_pickle(
    'kaggle/working/dis_features_2019df.pickle')

labs_feature_2019df = pd.read_pickle(
    'kaggle/working/labs_feature_2019_df.pickle')

prescr_features_2019df = pd.read_pickle(
    'kaggle/working/prescr_features_2019df.pickle')

In [67]:
test_df = train_labels_df[['member_id']].copy()

for df in [adm_features_2019df, dis_features_2019df, 
           labs_feature_2019df, prescr_features_2019df]:
    
    test_df = test_df.merge(
        df, 
        on = 'member_id', 
        how= 'left'
    )
    print(test_df.shape)
    
print(all(test_df.columns == train_df.columns))
test_df.to_pickle('kaggle/working/test_df.pickle')

(26559, 114)
(26559, 284)
(26559, 288)
(26559, 359)
True


In [68]:
train_labels_df = pd.read_csv(f'{data_path}train_labels.csv')

In [69]:
print(all(train_labels_df.member_id == train_df.member_id))
print(all(train_labels_df.member_id == test_df.member_id))

True
True


# Train

In [70]:
train_df.head(1)

,member_id,num_apprnc_in_hosptl,sum_readmissions,sum_admission_transfer,max_day_in_hosptl,mean_day_in_hosptl,adm_type_ER,adm_type_INP,adm_type_INP REHAB,adm_type_LTAC,...,rel_max_days_supply,mean_metric_quantity,std_metric_quantity,median_metric_quantity,max_metric_quantity,min_metric_quantity,rel_mean_metric_quantity,rel_median_metric_quantity,rel_max_metric_quantity,ratio_new_to_refill
0,c5xNoBb+QoCII8LRiNnScg==,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.25,39.428571,43.029724,14.0,100.0,0.0,0.537186,0.155556,0.274725,NaN


In [71]:
X_train, y_train, X_val, y_val =  iterative_train_test_split(
    train_df.drop('member_id', axis= 1).fillna(0).values,
    train_labels_df.drop('member_id', axis= 1).values,
    test_size = 0.15)

print(f'Train shape: {X_train.shape},Val shape {X_val.shape}')

Train shape: (22575, 358),Val shape (3984, 358)


In [72]:
def class_num_dis(x):
    df = pd.DataFrame([[i for i in (range(30))],
        [sum(x[:,i]) for i in range(30)]]).T
    df.columns = ['dis_clmn_number', 'num_dis']
    print(f'Zero classes:{df[df.num_dis == 0].dis_clmn_number.values}')
    return df

In [73]:
class_num_dis(y_train).head()

Zero classes:[]


,dis_clmn_number,num_dis
0,0,62
1,1,77
2,2,164
3,3,373
4,4,1033


In [74]:
class_num_dis(y_val).head()

Zero classes:[12]


,dis_clmn_number,num_dis
0,0,13
1,1,14
2,2,29
3,3,68
4,4,182


In [75]:
xgb_estimator = xgb.XGBClassifier(
    n_estimators = 300,
    eval_metric= 'logloss',
    use_label_encoder= False
) 

multilabel_model = MultiOutputClassifier(
    xgb_estimator,
)

multilabel_model.fit(
    X_train,
    y_train
)

print('Model trained')

/Users/artem/Documents/ML_projets/venvs/work_env/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/artem/Documents/ML_projets/venvs/work_env/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Model trained


In [76]:
dump(
    multilabel_model,
    'kaggle/working/model_emdng_ftrs.joblib')

['kaggle/working/model_emdng_ftrs.joblib']

In [77]:
val_proba_df = multilabel_model.predict_proba(
    X_val)

val_proba_df = np.transpose(
    [pred[:,1] for pred in val_proba_df])

print(val_proba_df.shape)

(3984, 30)


## Select threshold on validation set

In [78]:
def custom_auc_score(y_true, y_pred, thr):
    if pd.isnull(thr):
        return None
    lable_pred = [1 if p > thr else 0 for p in y_pred]
    try: 
        c_auc = roc_auc_score(y_true, lable_pred)
        return c_auc
    except Exception as e: 
        return None
    
def return_top_auc_row(
    class_number:int, 
    y_val_,
    val_proba_df_):
    '''return list with highest auc'''
    
    precision_, recall_, thresholds_ = precision_recall_curve(
        y_val_,
        val_proba_df_)

    pr_df = pd.DataFrame([
        precision_, 
        recall_, 
        thresholds_
    ]).T
    
    pr_df.columns= ['precision', 'recall', 'threshold']

    pr_df['custom_auc'] = pr_df['threshold'].apply(
        lambda x: custom_auc_score(
            y_val_,
            val_proba_df_,
            x))

    pr_df.sort_values(
        by= 'custom_auc',
        ascending= False, 
        inplace= True
    )
    
    res_li = [class_number] + list(pr_df.iloc[0])
    print(f'Auc:{res_li[-1]}')
    return res_li

In [79]:
ftr_df_li = []
for i in tqdm(range(30)): 
    cur_y_val, cur_prob_val = y_val[:,i], val_proba_df[:,i]
    ftr_df_li.append(
        return_top_auc_row(
            i,
            cur_y_val,
            cur_prob_val)
    )

  0%|          | 0/30 [00:00<?, ?it/s]

Auc:0.6377874203358967
Auc:0.6378373515653113
Auc:0.5952744234709447
Auc:0.543471729856396
Auc:0.7164144377453162
Auc:0.5880566801619433
Auc:0.6158166734666123
Auc:0.6580035702534351
Auc:0.642361111111111
Auc:0.6073500260824205
Auc:0.622829247472457
Auc:0.5819757559214364
Auc:None


/Users/artem/Documents/ML_projets/venvs/work_env/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


Auc:0.5
Auc:0.6920754579554973
Auc:0.6619674122052424
Auc:0.6984155935164609
Auc:0.570678756702559
Auc:0.7292577529232335
Auc:0.6696313241164362
Auc:0.662362971375965
Auc:0.6055669703430897
Auc:0.6162421481328086
Auc:0.7815980210265924
Auc:0.6583370170591201
Auc:0.5654641775983854
Auc:0.6418012108980826
Auc:0.6201642368773288
Auc:0.6494588389635947
Auc:0.6628755183210628


In [80]:
thr_df = pd.DataFrame(ftr_df_li)
thr_df.columns = ['dis_number', 'precision', 
                  'recall', 'threshold', 'top_auc']
print(thr_df.shape)
thr_df.head()

(30, 5)


,dis_number,precision,recall,threshold,top_auc
0,0,0.011390,0.384615,0.000128,0.637787
1,1,0.004988,0.928571,0.000010,0.637837
2,2,0.009050,0.965517,0.000008,0.595274
3,3,0.019591,0.661765,0.000046,0.543472
4,4,0.098592,0.769231,0.000585,0.716414


In [81]:
np.mean(thr_df.top_auc)

0.635623304670991

In [ ]:
thr_df.to_pickle(
    'kaggle/working/thr_df.pickle'
)

## Training on all train DF

In [ ]:
# n_estimators = 10 - just to upload to GitHub
xgb_estimator = xgb.XGBClassifier(
    n_estimators = 10,
    eval_metric='logloss',
    use_label_encoder=False,
) 

full_multilabel_model = MultiOutputClassifier(
    xgb_estimator
)

full_multilabel_model.fit(
    train_df.drop('member_id', axis= 1).fillna(0).values,
    train_labels_df.drop('member_id', axis= 1).values,
)

print('Model trained')

In [ ]:
dump(
    full_multilabel_model,
    'kaggle/working/full_model_no_embdngs.joblib') 

## Make prediction

In [ ]:
test_proba = full_multilabel_model.predict_proba(
    test_df.drop('member_id', axis= 1).fillna(0).values
)

test_proba = np.transpose(
    [pred[:,1] for pred in test_proba])

In [ ]:
test_proba_df = pd.DataFrame(test_proba)
test_proba_df.columns = list(train_labels_df.columns)[1:]
test_proba_df.head(1)

In [ ]:
res_df = pd.DataFrame()
for clmn, thr_ in zip(test_proba_df.columns, thr_df.threshold): 
    res_df[clmn] = test_proba_df[clmn].apply(
        lambda x: 1 if x > thr_ else 0)

In [ ]:
res_df['member_id'] = test_df.member_id.values
res_df = res_df[train_labels_df.columns].copy()
print(res_df.shape)
res_df.head()

In [ ]:
res_df.to_csv(
    'kaggle/working/emb_res_df.csv', 
    index= False
)